In [ ]:
import requests
import json
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
from lxml import html
import pandas as pd
from urllib.request import Request, urlopen

In [ ]:
from fake_useragent import UserAgent
ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]

In [ ]:
def getProx():
    proxies_req = Request('https://www.sslproxies.org/')
    proxies_req.add_header('User-Agent', ua.random)
    proxies_doc = urlopen(proxies_req).read().decode('utf8')
    soup = BeautifulSoup(proxies_doc, 'html.parser')
    proxies_table = soup.find(id='proxylisttable')
         # Save proxies in the array
    global proxies
    proxies=[]
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append({
        'ip':   row.find_all('td')[0].string,
        'port': row.find_all('td')[1].string
      })
    #print(len(proxies))
    #print(proxies)

In [ ]:
getProx()

In [ ]:
import random
proxy_index = random.randint(0, len(proxies) - 1)
proxy = proxies[proxy_index] 
req = Request('http://icanhazip.com')
req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

In [ ]:
df =pd.read_csv("altmetricIDS.csv", header=None)

In [ ]:
df.columns = ['Altmetrics_Id']
df.head()

In [ ]:
df["youtube_links"] = ""
df.head()

In [ ]:
ids = np.array(df['Altmetrics_Id'])
ids[5255]

In [ ]:
alt_param  = {"key" : ""}

In [ ]:
all_alt_data = requests.get("https://api.altmetric.com/v1/id/1969481", alt_param )

In [ ]:
alt_data = json.loads(all_alt_data.text)
alt_data.keys()

In [ ]:
y_links = []
html = requests.get('https://www.altmetric.com/details/' + str(403270) + '/video', alt_param).text
bs = BeautifulSoup(html)
possible_links = bs.find_all('a')
for link in possible_links:
    if link.has_attr('href') and 'youtube' in link.attrs['href']:
        y_links.append(link.attrs['href'])
y_links

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common import exceptions

In [ ]:
chrome = webdriver.Chrome('C:/ProgramData/chocolatey/bin/chromedriver.exe')

In [ ]:
chrome.get('https://www.altmetric.com/details/403270/video')

In [ ]:
def get_all_pages(main_link):
    myset= set()
    chrome.get(main_link)
    elements = chrome.find_elements_by_xpath("//a[@href]")
    for elem in elements:
        if "video/page:" in elem.get_attribute("href"):
            myset.add(elem.get_attribute("href"))
    return myset

In [ ]:
def get_youtube_ids(link):
    y_links = []
    chrome.get(link)
    elements = chrome.find_elements_by_xpath("//a[@href]")
    for elem in elements:
        if "youtube" in elem.get_attribute("href"):
            y_links.append(elem.get_attribute("href")[32:])
    return y_links

In [ ]:
from time import sleep

In [ ]:
file = open('youtube.csv', 'w')
for i in range(len(ids)):
    try:
        getProx()
        proxy_index = random.randint(0, len(proxies) - 1)
        proxy = proxies[proxy_index] 
        req = Request('http://icanhazip.com')
        req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')
        sleep(0.5)
        y_links = get_youtube_ids('https://www.altmetric.com/details/' + str(ids[i]) + '/video')
        myset = get_all_pages('https://www.altmetric.com/details/' + str(ids[i]) + '/video')
        for link in myset:
            y_links.extend(get_youtube_ids(link))
        df.at[i, 'youtube_links'] = y_links
        file.write(str(ids[i]) + ",")
        file.write(" ".join(y_links) + "\n")
        
    except exceptions.StaleElementReferenceException:
        sleep(0.5)
        continue
        
        
    try:
        my_ip = urlopen(req).read().decode('utf8')        
        print('#' + str(n) + ': ' + my_ip)

    except:  
        # If error, delete this proxy and find another one
        #global proxies
        del proxies[proxy_index]
        print('proxy deleted')
        proxy_index = random.randint(0, len(proxies) - 1)
        proxy = proxies[proxy_index]

In [ ]:
file.close()

In [ ]:
len(df.loc[4, 'youtube_links'])

In [ ]:
df.to_csv("youtube_links.csv")

In [ ]:
alt_data['details_url']

In [ ]:
def add_video(alt_id):
    try:
        response = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))
        if 'video' in dict(json.loads(response.content)):
            return dict(json.loads(response.content))['video']
    except:
        return ''

In [ ]:
param = {"key" : ""}

In [ ]:
url_content = "https://www.googleapis.com/youtube/v3/videos?part=snippet&key=&id="

In [ ]:
url = "https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics"

In [ ]:
ids[7132]

In [ ]:
for alt_id in ids[7133:7433]:
    mylist = []
    alt_req = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))
    alt_rs= json.loads(alt_req.text)
    mylist.append({str(alt_id) : alt_rs})
    y_links = []
    html = requests.get('https://www.altmetric.com/details/' + str(alt_id) + '/video').text
    bs = BeautifulSoup(html)
    possible_links = bs.find_all('a')
    for link in possible_links:
        if link.has_attr('href') and 'youtube' in link.attrs['href']:
            y_links.append(link.attrs['href'])
    youtube_data = []
    for link in y_links:
        param["id"] = link[32:]
        response_statistics = requests.get(url, param)
        print(response_statistics.status_code)
        json_statistics = json.loads(response_statistics.text)
        youtube_data.append(json_statistics)
    mylist.append({'youtube':youtube_data})
    file = open(str(alt_id) + ".txt", "w+")
    json.dump(mylist, file)
    file.close()

In [ ]:
import os
path = "C:/Users/srika/Youtube_dataSet"
files = os.listdir(path)
len(path)

In [ ]:
for i in range(len(files)):
    files[i] = os.path.abspath(path + "/" + files[i])

In [ ]:
len("C:\\Users\\srika\\Youtube_dataSet\\")

In [ ]:
df_ = pd.DataFrame()
col = ['cited_by_fbwalls_count', 'cited_by_feeds_count', 'cited_by_gplus_count', 'cited_by_msm_count', 
       'cited_by_posts_count', 'cited_by_rdts_count', 'cited_by_tweeters_count', 'cited_by_videos_count']

In [ ]:
stats = ['viewCount', 'likeCount',  'dislikeCount', 'favoriteCount','commentCount']

In [ ]:
i=0
for file in files:
    json_file = open(file, "r")
    data = json.load(json_file)
    df_.loc[i, "altmetric_id"] = file[31:-4]
    for c in col:
        try:
            df_.loc[i, c] = data[0][file[31:-4]][c]
        except KeyError:
            df_.loc[i, c] = 0
    i += 1
    json_file.close
    
df_.head() 

In [ ]:
df_videos = pd.DataFrame(columns = stats, dtype=object)

In [ ]:
for i in range(len(files[:2])):
    json_file = open(files[i], "r")
    data = json.load(json_file)
    df_videos.loc[i, 'altmetric_id'] = files[i][31:-4]
    for stat in stats:
        mylist = []
        for j in range(len(data[1]['youtube'])):
            try:
                mylist.append(int(data[1]['youtube'][j]['items'][0]['statistics'][stat]))
            except IndexError:
                mylist.append(0)
            except KeyError:
                mylist.append(0)
        print(stat, mylist)
        df_videos.at[i, stat] = mylist
    json_file.close()
df_videos.head()

In [ ]:
df_.to_csv("altmetrics_data.csv")

In [ ]:
df_videos.to_csv("videos.csv") 

In [ ]:
y_link = url_statistics +"GSqdhz--yR8"
rs_st = requests.get(y_link)
rs_st_d = json.loads(rs_st.text)
rs_st_d

In [ ]:
y_content = url_content + "GSqdhz--yR8"
rs_ct = requests.get(y_link)
rs_ct_d = json.loads(rs_st.text)
rs_ct_d

In [ ]:
social_media_columns = ['cited_by_fbwalls_count', 'cited_by_gplus_count', 'cited_by_msm_count', 'cited_by_posts_count', 
                        'cited_by_tweeters_count', 'cited_by_videos_count', 'cited_by_wikipedia_count', 
                        'cited_by_accounts_count',]

In [ ]:
cols = ['title', 'doi']

In [ ]:
for i in range(len(ids[:100])):
    rs = requests.get("https://api.altmetric.com/v1/id/" + str(ids[i]))
    if (rs.status_code == 200):
        rs_json = json.loads(rs.text)
        for col in cols:
            try:
                df.loc[i, col] = rs_json[col]
            except KeyError:
                df.loc[i, col] = 0

In [ ]:
df.head()

In [ ]:
df[:100].to_csv(path_or_buf = ".\\sample_dataset.csv", index=False)

In [ ]:
url_comments = "https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=_qUhIZcxNvE&key="

In [ ]:
response = requests.get(url)
print(response.status_code)

In [ ]:
json_data = json.loads(response.text)
json_data.keys()

In [ ]:
json_data["items"][2]['snippet']['topLevelComment']['snippet']

In [ ]:
lc = 0
dc = 0
for j in len(json_data['items']):
    lc + = json_data['items'][j]["likeCount"]
    dc + = json_data['items'][j]["likeCount"]

In [ ]:
# add author information
def add_all_info(alt_id):
    # create the dict
    result = defaultdict(dict)
    try:
        # make the request
        response = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))

        # add the values to the dict
        if 'authors' in dict(json.loads(response.content)):
            result['author_count'] = len(dict(json.loads(response.content))['authors'])
            result['authors'] = ', '.join(dict(json.loads(response.content))['authors'])
        else:
            result['author_count'] = ''
            result['authors'] = ''

         # add the values to the dict
        if 'doi' in dict(json.loads(response.content)):
            result['doi'] = dict(json.loads(response.content))['doi']
        else:
            result['doi'] = ''

        # add the abstract
        if 'abstract' in dict(json.loads(response.content)):
            result['abstract'] = dict(json.loads(response.content))['abstract']
        else:
            result['abstract'] = ''

        # add the pubdate
        if 'published_on' in dict(json.loads(response.content)):
            result['pub_date'] = dict(json.loads(response.content))['published_on']
        else:
            result['pub_date'] = ''

        # add the twitter posts count as of 2018/19
        if 'cited_by_tweeters_count' in dict(json.loads(response.content)):
            result['twitter_count_y18'] = dict(json.loads(response.content))['cited_by_tweeters_count']
        else:
            result['twitter_count_y18'] = ''

        # add the title of the article
        if 'title' in dict(json.loads(response.content)):
            result['title'] =  dict(json.loads(response.content))['title']
        else:
            result['title'] = ''

        # return the result
        return result
    except:
        return result

In [ ]:
r = add_all_info(702709 )

In [ ]:
r.keys()

In [ ]:
rs_ = requests.get("https://api.altmetric.com/v1/id/" + str(2394296))

In [ ]:
rs_js = json.loads(rs_.text)

In [ ]:
rs_js.keys()

In [ ]:
fl = open("temp.txt", 'w+')

In [ ]:
json.dump(rs_js, fl)

In [ ]:
rs_js['cited_by_accounts_count']

In [ ]:
rs_js['cited_by_videos_count']

In [ ]:
response_statistics = requests.get(url_statistics)
print(response.status_code)

In [ ]:
json_statistics = json.loads(response_statistics.text)
json_statistics.keys()

In [ ]:
json_statistics['items']

In [ ]:
cookies = {"LOGIN_INFO": "AFmmF2swRQIhAIEmerJsgJPTtDcSlfROZB4nA7CDB8gni2t6wihB-r6wAiArbacirmZ0s6ofJZ5ORMIfI9dKOj-pXVAHPIh22PO6qQ:QUQ3MjNmd0VYNm1TVmhVUmdRM2Z5U3RGdy1tQTRnUkN4clJIdTJiakVSYmdvd0F5bVNKRDNtZndtemtsM1FBQkM3NzduRHFKaDhVdGlvelQwM3UtX0xKc2tVemRLT3NSTGgyUkhwdkJLbHZHWlQyZUpVaXcyb1VfdnR2c1pESXM2eWlUMGczbVl1Y0VPZTVHQklJMFdRQnNWOWZSX2pVMU16ZkRPN21NS2lHdmhReDNQQWVYZERJ"}

In [ ]:
you_req = requests.get("https://www.youtube.com/watch?v=zAi6Qe0qiIs", params = param, cookies=cookies).text

In [ ]:
bs = BeautifulSoup(you_req)
possible_links = bs.find_all('script')

In [ ]:
possible_links[]

In [ ]:
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
chrome = webdriver.Chrome('C:/ProgramData/chocolatey/bin/chromedriver.exe')
chrome.get('https://www.youtube.com/watch?v=zAi6Qe0qiIs')

In [ ]:
elements = chrome.find_elements_by_xpath('//yt-formatted-string[@class = "style-scope ytd-toggle-button-renderer style-text"]')

In [ ]:
mylist = []
for element in elements:
    mylist.append(element.get_attribute("aria-label"))
mylist

In [ ]:
for i in range(len(mylist)):
    mylist[i] = int(mylist[i].replace(",", "").split()[0])
mylist

In [ ]:
view_elements = chrome.find_elements_by_xpath('//div[@id = "count"]')

In [ ]:
for v in view_elements:
    print(v.find_elements_by_class_name("view-count style-scope yt-view-count-renderer"))

In [ ]:
wait = WebDriverWait(chrome, 20)
v_title = chrome.find_element_by_xpath('//h1[@class="title style-scope ytd-video-primary-info-renderer"]')
v_title.text

In [ ]:
v_title

In [ ]:
v_description =  chrome.find_element_by_xpath('//div[@id = "description" and @class="style-scope ytd-video-secondary-info-renderer"]')

In [ ]:
v_description.text

In [ ]:
v_likes =  wait.until(EC.presence_of_element_located(
                                         (By.CSS_SELECTOR,"div#top-level-buttons yt-formatted-string"))).text


In [ ]:
v_likes

In [ ]:
count =chrome.find_element_by_xpath('//div[@id = "count" and @class="style-scope ytd-video-primary-info-renderer"]')
count.text

In [ ]:
count = int(count.replace(",", "").split()[0])
count

In [ ]:
n_comments = chrome.find_element_by_xpath('//h2[@id="count" and @class="style-scope ytd-comments-header-renderer"]')

In [ ]:
n_comments.text

In [ ]:
n_comm = int(n_comments.text.replace(",", "").split()[0])
n_comm

In [ ]:
channel_id = chrome.find_element_by_xpath('//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]')

In [ ]:
c_id = channel_id.get_attribute('href')
c_id[32:]

In [ ]:
comment_elements = chrome.find_elements_by_xpath('//ytd-comments')
for c in comment_elements:
    for element in c.find_elements_by_tag_name('yt-formatted-string'):
        print (element.text)

In [ ]:
len('https://www.youtube.com/channel/')